In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
import math
from sklearn.metrics import hamming_loss,confusion_matrix

sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')

from quantum_kernel.code.utils import get_dataset,precomputed_kernel_GridSearchCV
from quantum_kernel.code.visualization_utils import aggregate_folder
from sklearn.svm import SVC

font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 18}

rc('font', **font)

tqdm.pandas()

In [ ]:
# Get kernel results for the different ml models and dataset.d
dfs = {}
dataset_name='plasticc'
folders = [
    ("/mnt/c/Users/lslat/Data/QK_project/results/hproj_HZZMulti/"+dataset_name,'HZZMulti'),
]

for folder, kernel_name in folders:
    if dataset_name in dfs:
        temp = aggregate_folder(folder, dataset_name, kernel_name=kernel_name)[
            dataset_name]
        dfs[dataset_name] = pd.concat([dfs[dataset_name], temp])
    else:
        dfs[dataset_name] = aggregate_folder(
            folder, dataset_name, kernel_name=kernel_name)[dataset_name]

save_folder='/mnt/c/Users/lslat/Data/QK_project/figs/'


In [ ]:
df = dfs[dataset_name]
